# MLP Classifier

In [158]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score

In [159]:
from sklearn.neural_network import  MLPClassifier

In [160]:
# import du csv
data = pd.read_csv('csv_correction.csv')
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non


### préparation des datas

In [161]:
indexDrop = data[((data['fk_arb_etat'] == 'ABATTU') | (data['fk_arb_etat'] == 'EN PLACE') | (data['fk_arb_etat'] =='REMPLACÉ') | (data['fk_arb_etat'] == 'SUPPRIMÉ') )].index
data.drop(indexDrop, inplace = True)

In [162]:
data["fk_arb_etat"].value_counts()


fk_arb_etat
Essouché        165
Non essouché     27
Name: count, dtype: int64

In [163]:
list_col = ['fk_stadedev', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech', 'fk_arb_etat']
for col in list_col:
  encoder = preprocessing.LabelEncoder()
  encoded = encoder.fit_transform(data[col])
  #print(encoder.categories_)
  data[col] = encoded

In [164]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

### split des données

In [165]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data[all_cols_list], data["fk_arb_etat"], test_size=0.2, random_state=42, stratify=data["fk_arb_etat"])

### create model instance

In [166]:
mlp = MLPClassifier(hidden_layer_sizes=(20, 50),validation_fraction=0.2, learning_rate="adaptive", random_state=55) # col*2, *2

### Fit

In [167]:
mlp.fit(X_train, y_train)

C:\Users\lebou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              random_state=55, validation_fraction=0.2)

### Validation

In [168]:
confusion_matrix(y_train, mlp.predict(X_train), normalize="true")

array([[0.98473282, 0.01526718],
       [0.36363636, 0.63636364]])

### Export du modèle

In [169]:
import pickle

In [170]:
with open('mlp_model_1.pkl', 'wb') as f:
    pickle.dump(mlp, f, protocol=pickle.HIGHEST_PROTOCOL)

In [174]:
d = pd.read_json("res_df.json")

In [175]:
d

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,storm_predictions
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37,EN PLACE,Jeune,semi libre,...,Alignement,Non,15,5,0,QUERUB,VILLE,Feuillu,Non,0
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160,EN PLACE,Adulte,semi libre,...,Groupe,Non,50,10,0,PINNIGnig,VILLE,Conifère,Non,0
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116,REMPLACÉ,Adulte,semi libre,...,Alignement,Non,30,10,0,ACEPSE,VILLE,Feuillu,Non,1
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150,EN PLACE,Adulte,semi libre,...,Groupe,Non,50,2,0,ACEPLA,VILLE,Feuillu,Non,1
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170,Essouché,Adulte,réduit,...,Isolé,Non,40,2,0,SALBAB,VILLE,Feuillu,Non,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,5,1,0,ALBJUL,VILLE,Feuillu,Non,0
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,3,1,0,ALBJULcho,VILLE,Feuillu,Non,0
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Isolé,Non,5,1,0,CEDLIB,VILLE,Conifère,Non,0
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15,EN PLACE,Jeune,Libre,...,Alignement,Non,5,1,0,FAGSYLdawgol,VILLE,Feuillu,Non,0


In [176]:
d["storm_predictions"].value_counts()

storm_predictions
0    6516
1     893
Name: count, dtype: int64